In [2]:
import numpy as np
from qiskit import QuantumCircuit, transpile, assemble
from qiskit_aer import Aer
from qiskit.visualization import array_to_latex

def initialize_qubits(amplitudes):
    """
    Creates a quantum circuit that initializes two qubits to a given normalized state.
    :param amplitudes: List of four complex numbers representing the desired state amplitudes.
    :return: QuantumCircuit object with initialized qubits.
    """
    assert len(amplitudes) == 4, "Input must have four amplitudes."
    assert np.isclose(np.sum(np.abs(amplitudes)**2), 1), "Input state must be normalized."
    
    a0, a1, a2, a3 = amplitudes
    
    # Compute angles based on given formulas
    theta1 = 2 * np.arccos(np.sqrt(a0**2 + a2**2))
    theta2 = 2 * np.arctan2(a3, a1)
    theta3 = 2 * np.arctan2(a2, a0)
    
    # Create quantum circuit with 2 qubits
    qc = QuantumCircuit(2)
    
    # Apply Ry rotation on qubit 0
    qc.ry(theta1, 0)
    
    # Controlled Ry rotation on qubit 1 (controlled by qubit 0)
    qc.cry(theta2, 0, 1)
    
    # Apply X gate to qubit 0 (negation)
    qc.x(0)
    
    # Another controlled Ry rotation
    qc.cry(theta3, 0, 1)
    
    # Apply X gate again to qubit 0 to revert
    qc.x(0)
    
    return qc

def check_initialization(qc):
    """
    Uses the statevector simulator to check the resulting quantum state.
    """
    simulator = Aer.get_backend('statevector_simulator')
    transpiled_qc = transpile(qc, simulator)
    result = simulator.run(transpiled_qc).result()
    statevector = result.get_statevector()
    
    print("Resulting statevector:")
    display(array_to_latex(statevector, prefix="|ψ⟩ = "))
    return statevector

def end_matrix(qc):
    """
    Uses the unitary simulator to extract the full matrix representation of the circuit.
    """
    simulator = Aer.get_backend('unitary_simulator')
    transpiled_qc = transpile(qc, simulator)
    result = simulator.run(transpiled_qc).result()
    unitary = result.get_unitary()
    return np.real(unitary)

def gate_matrix(gate, nqubits):
    """
    Returns the matrix representation of a single gate in a quantum circuit.
    """
    qc = QuantumCircuit(nqubits)
    qc.append(gate.operation, qargs=[q.index for q in gate.qubits])
    return end_matrix(qc)

def gate_matrices(qc):
    """
    Returns the list of unitary matrices for each gate in the circuit.
    """
    matrices = []
    for gate in qc.data:
        matrices.append(gate_matrix(gate, qc.num_qubits))
    return matrices

def step_matrices(qms):
    """
    Computes the cumulative product of matrices to show stepwise transformations.
    """
    cumulative_matrices = []
    current_matrix = np.eye(qms[0].shape[0])
    for matrix in qms:
        current_matrix = np.dot(matrix, current_matrix)
        cumulative_matrices.append(current_matrix)
    return cumulative_matrices

# Example test
amplitudes = np.sqrt(np.array([1/2, 1/4, 1/6, 1/12]))
qc = initialize_qubits(amplitudes)
qc.draw('mpl')

# Check initialization
statevector = check_initialization(qc)

# Check matrix semantics
unitary_matrix = end_matrix(qc)
print("Full unitary matrix:")
print(unitary_matrix)

# Check individual gate matrices
gate_matrices_list = gate_matrices(qc)
cumulative_matrices = step_matrices(gate_matrices_list)

# Verify if cumulative matrices match full unitary
assert np.allclose(cumulative_matrices[-1], unitary_matrix), "Mismatch in cumulative matrix product!"

Resulting statevector:


<IPython.core.display.Latex object>

Full unitary matrix:
[[ 0.70710678 -0.5        -0.40824829  0.28867513]
 [ 0.5         0.70710678 -0.28867513 -0.40824829]
 [ 0.40824829 -0.28867513  0.70710678 -0.5       ]
 [ 0.28867513  0.40824829  0.5         0.70710678]]


AttributeError: 'Qubit' object has no attribute 'index'